In [ ]:
import boto3

In [ ]:
# まずLambdaが使用するロールを作る
iam = boto3.resource('iam')

In [ ]:
# ロールの「信頼ポリシー」(Trust policy)を定義。
# 「信頼ポリシー」には、「AWSのサービスであるLambdaが、このロールを使ってよい」と書く。
# 「ロールを使う」＝ AssumeRole
policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [ ]:
# ロールを作成。「信頼ポリシー」(Trust policy)をアタッチ。
import json

try:
    role = iam.create_role(RoleName='role_for_lambda_sample', AssumeRolePolicyDocument=json.dumps(policy_doc))
    print('ロールを定義しました: ', 'role_for_lambda_sample')
except iam.meta.client.exceptions.EntityAlreadyExistsException:
    print('ロールは定義済みです: ', 'role_for_lambda_sample')


In [ ]:
# ロールに「アクセス権限ポリシー」(Permissions policy)をアタッチ。
# アクセス権限ポリシーとは、このLambda関数のプログラムが、何をしてよいか、という設定。
# アクセス権限ポリシーとして、CloudWatch Logsへのフルアクセス（書き込みを含む）を許可。

# role.attach_policy(PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')
role.attach_policy(PolicyArn='arn:aws:iam::aws:policy/CloudWatchLogsFullAccess')

In [ ]:
# Lambdaを操作するためのオブジェクトを作成。
client = boto3.client('lambda') # lambda はPythonの予約語なので、ここではclientとしている。

In [ ]:
# Lambda関数コードを作成
code = '''
import json

def lambda_handler(event, context):
    print('hello!!こんにちは!!! このメッセージはCloudWatch Logsに出力されます')
    response_data = {'message':'さようなら..!このメッセージはレスポンスに含まれます'}
    return {
        "statusCode": 200,
        #"body": json.dumps(response_data)
        'body': response_data
    }
'''
print(code)

In [ ]:
# Lambda関数コードをapp.pyに書き込み、さらにapp.zipとして圧縮する

import zipfile
import os.path
import datetime

str_info = zipfile.ZipInfo()
current_time = datetime.datetime.now()

str_info.filename = 'app.py'
str_info.compress_type = zipfile.ZIP_DEFLATED
str_info.date_time = (current_time.year, current_time.month, current_time.day, 
                      current_time.hour, current_time.minute, current_time.second)
str_info.external_attr = 0o777 << 16
with zipfile.ZipFile('app.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.writestr(str_info, code)


In [ ]:
# Lambda関数名を定義
func_name = 'lambda_sample'

In [ ]:
# 同名の関数は定義できないので、もし関数があれば先に削除しておく。
try:
    client.delete_function(FunctionName=func_name)
    print('削除しました: ', func_name)
except client.exceptions.ResourceNotFoundException:
    print('関数は存在しません: ', func_name)


In [ ]:
from pprint import pprint

# Lambda関数を作成（ZIPをアップロード）
with open('app.zip', 'rb') as app:
    func = client.create_function(
        FunctionName=func_name, # 関数名
        Runtime='python3.6', # ランタイム（言語）
        Role=role.arn, # ロールのARN
        Handler='app.lambda_handler', # app.py の lambda_handler 関数をハンドラ（起点）とする
        Code={
            'ZipFile': app.read() # ZIPファイルをアップロード
        },
        Description='description', # 説明文
        Timeout=60, # 関数の最長実行時間
        MemorySize=128 # 関数が使用できる最大メモリ量
    )
    pprint(func)

In [ ]:
# Lambda関数を呼び出す
import json
from pprint import pprint
result = client.invoke(
    FunctionName=func_name, 
    InvocationType='RequestResponse')

In [ ]:
pprint(result)

In [ ]:
# Lambda関数が出力したJSONを表示
json.loads(result['Payload'].read())